In [ ]:
!conda create --name rl python=3.10.12
!conda activate rl

!git clone https://github.com/Unity-Technologies/ml-agents
    
%cd ml-agents
!pip install -e ./ml-agents-envs
!pip install -e ./ml-agents
!pip install gdown

!apt-get install -y git-lfs

!mkdir ./training-envs-executables

!gdown https://drive.google.com/uc?id=1KuqBKYiXiIcU4kNMqEzhgypuFP5_45CL -O ./training-envs-executables/SoccerTwos.zip

!unzip -d ./training-envs-executables/ ./training-envs-executables/SoccerTwos.zip

!chmod -R 755 ./training-envs-executables/SoccerTwos.x86_64

import yaml
from huggingface_hub import notebook_login

config = {
    "behaviors": {
        "SoccerTwos": {
            "trainer_type": "poca",
            "hyperparameters": {
                "batch_size": 2048,
                "buffer_size": 20480,
                "learning_rate": 0.003,
                "beta": 0.005,
                "epsilon": 0.2,
                "lambd": 0.95,
                "num_epoch": 3,
                "learning_rate_schedule": "constant"
            },
            "network_settings": {
                "normalize": False,
                "hidden_units": 512,
                "num_layers": 2,
                "vis_encode_type": "simple"
            },
            "reward_signals": {
                "extrinsic": {
                    "gamma": 0.99,
                    "strength": 1.0
                }
            },
            "keep_checkpoints": 5,
            "max_steps": 5000000,
            "time_horizon": 1000,
            "summary_freq": 10000,
            "self_play": {
                "save_steps": 50000,
                "team_change": 50000,
                "swap_steps": 2000,
                "window": 10,
                "play_against_latest_model_ratio": 0.5,
                "initial_elo": 1200.0
            }
        }
    }
}

with open("./config/poca/SoccerTwos.yaml", "w") as f:
    yaml.dump(config, f)
    
    
!mlagents-learn ./config/poca/SoccerTwos.yaml --env=./training-envs-executables/SoccerTwos --run-id="soccer_twos" --no-graphics

notebook_login()

!mlagents-push-to-hf  --run-id="soccer_twos" --local-dir="./results/soccer_twos" --repo-id="jaymanvirk/poca_soccer_twos" --commit-message="initial commit"